In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
filtro_enem = [
    "NU_INSCRICAO", # Identificador único da inscrição
    "TP_ESCOLA",  # Tipo de escola do Ensino Médio (Público, Privado)
    "TP_ENSINO", #Ensino médio ou especial
    "CO_MUNICIPIO_ESC", #Código de identificação da escola
    "SG_UF_ESC",  # Sigla da Unidade da Federação da escola
    "TP_DEPENDENCIA_ADM_ESC", #Dependência administrativa
    "TP_LOCALIZACAO_ESC", #localização da escola
    "NU_NOTA_CN", #nota da prova de Ciências da Natureza
    "NU_NOTA_CH", #nota da prova de Ciências Humanas
    "NU_NOTA_LC", #nota da prova de Linguagens e Códigos
    "NU_NOTA_MT", #nota da prova de Matemática
    "NU_NOTA_REDACAO", #nota da prova de Redação
    "Q001", #educação do pai
    "Q002", #educação da mãe
    "Q006", #renda familiar
    "Q024", #possui computador em casa
    "Q025" #possui internet em casa
    ]

In [3]:
#base do enem 2022
enem = pd.read_csv('dados/MICRODADOS_ENEM_2022.csv', usecols=filtro_enem, sep=';', encoding='latin1')

In [4]:
#calcula média para enem 2022
enem['MEDIA'] = (enem['NU_NOTA_CN'] + enem['NU_NOTA_CH'] + enem['NU_NOTA_LC'] + enem['NU_NOTA_MT'] + enem['NU_NOTA_REDACAO']) / 5

In [7]:
#INSE
inse = pd.read_excel('dados/INSE_2021_escolas.xlsx', usecols=['ID_ESCOLA', 'INSE_CLASSIFICACAO'])

In [10]:
filtro_censo = [
    "CO_ENTIDADE", #Código da escola
    'IN_LABORATORIO_INFORMATICA', #Possui laboratório de informática
    'IN_BIBLIOTECA' #Possui biblioteca
    ]

#censo escolar educação básica
censo_escolar = pd.read_csv('dados/microdados_ed_basica_2022.csv', usecols=filtro_censo, sep=';', encoding='latin1')

In [13]:
#Características dos docentes

docentes = pd.read_excel('dados/formacao_docentes.xlsx')

tratamento

In [5]:
enem.dropna(subset='MEDIA', inplace=True) #tira todas as linhas que não tem média

In [6]:
dict_q001_2 = {
    'H': -1, #não sabe
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'G': 6
}

In [7]:
enem['Q001'] = enem['Q001'].map(dict_q001_2)
enem['Q002'] = enem['Q002'].map(dict_q001_2)

In [8]:
#criar uma nova coluna chamada EDUCACAO_PAI_MAE. Verifica para cada linha quem possui maior educação, se é o pai ou a mãe e atribuir o valor a nova coluna. As categorias são do A ao G, sendo A a menor e G a maior.

enem['EDUCACAO_PAI_MAE'] = enem[['Q001', 'Q002']].max(axis=1)


In [9]:
dict_escolaridade_pais = {
    -1: 'H',
    0: 'A', 
    1: 'B', 
    2: 'C', 
    3: 'D', 
    4: 'E', 
    5: 'F', 
    6: 'G' 
}

In [10]:
enem['EDUCACAO_PAI_MAE'] = enem['EDUCACAO_PAI_MAE'].map(dict_escolaridade_pais)

In [11]:
enem[['Q001', 'Q002', 'EDUCACAO_PAI_MAE']]

,Q001,Q002,EDUCACAO_PAI_MAE
2,4,5,F
3,2,0,C
7,4,4,E
9,4,4,E
11,4,4,E
...,...,...,...
3476095,4,4,E
3476097,-1,2,C
3476098,-1,4,E
3476099,2,1,C


infraestrutura

In [12]:
#se o aluno respondeu B, C, D e E na Q024 e B na Q025, então ele possui computador e internet em casa e recebe 1, caso contrário recebe 0

enem['COMPUTADOR_INTERNET'] = False

enem.loc[(enem['Q024'].isin(['B', 'C', 'D', 'E'])) & (enem['Q025'] == 'B'), 'COMPUTADOR_INTERNET'] = True

enem['COMPUTADOR_INTERNET'].value_counts()

COMPUTADOR_INTERNET
True     1341406
False    1003417
Name: count, dtype: int64

In [13]:
enem[['Q024', 'Q025', 'COMPUTADOR_INTERNET']]

,Q024,Q025,COMPUTADOR_INTERNET
2,A,B,False
3,B,B,True
7,A,B,False
9,A,B,False
11,A,B,False
...,...,...,...
3476095,A,B,False
3476097,B,B,True
3476098,A,B,False
3476099,A,A,False


Tipo de escola

In [14]:
dict_tp_escola = {
    1: 'Não respondeu',
    2: 'Pública',
    3: 'Privada'
}

In [15]:
enem['TP_ESCOLA'] = enem['TP_ESCOLA'].map(dict_tp_escola)

Tipo de ensino

In [16]:
dict_ensino = {
    1: 'Regular',
    2: 'Especial'
}

In [17]:
enem['TP_ENSINO'] = enem['TP_ENSINO'].map(dict_ensino)

In [18]:
tp_dependencia_adm_esc = {
    1: 'Federal',
    2: 'Estadual',
    3: 'Municipal',
    4: 'Privada'
}

In [19]:
enem['TP_DEPENDENCIA_ADM_ESC'] = enem['TP_DEPENDENCIA_ADM_ESC'].map(tp_dependencia_adm_esc)

Localização

In [20]:
dict_localizacao_esc = {
    1: 'Urbana',
    2: 'Rural'
}

In [21]:
enem['TP_LOCALIZACAO_ESC'] = enem['TP_LOCALIZACAO_ESC'].map(dict_localizacao_esc)

Modelo

In [22]:
y = pd.to_numeric(enem['MEDIA'])

X = enem[['TP_ESCOLA', 'TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'EDUCACAO_PAI_MAE', 'COMPUTADOR_INTERNET']]

In [36]:
X_dummies = pd.get_dummies(X)

In [37]:
X_dummies.columns

Index(['COMPUTADOR_INTERNET', 'TP_ESCOLA_Não respondeu', 'TP_ESCOLA_Privada',
       'TP_ESCOLA_Pública', 'TP_ENSINO_Especial', 'TP_ENSINO_Regular',
       'TP_DEPENDENCIA_ADM_ESC_Estadual', 'TP_DEPENDENCIA_ADM_ESC_Federal',
       'TP_DEPENDENCIA_ADM_ESC_Municipal', 'TP_DEPENDENCIA_ADM_ESC_Privada',
       'TP_LOCALIZACAO_ESC_Rural', 'TP_LOCALIZACAO_ESC_Urbana',
       'EDUCACAO_PAI_MAE_A', 'EDUCACAO_PAI_MAE_B', 'EDUCACAO_PAI_MAE_C',
       'EDUCACAO_PAI_MAE_D', 'EDUCACAO_PAI_MAE_E', 'EDUCACAO_PAI_MAE_F',
       'EDUCACAO_PAI_MAE_G', 'EDUCACAO_PAI_MAE_H'],
      dtype='object')

In [38]:
X_dummies.drop(columns=['TP_ESCOLA_Não respondeu', 'TP_ESCOLA_Pública', 'TP_ENSINO_Regular', 'TP_DEPENDENCIA_ADM_ESC_Municipal', 'TP_LOCALIZACAO_ESC_Urbana', 'EDUCACAO_PAI_MAE_H'], inplace=True)

In [39]:
# Adicionar uma coluna de intercepto aos dados
X_dummies = sm.add_constant(X_dummies)

In [40]:
X_dummies = X_dummies.astype('int64')

In [41]:
# Ajustar o modelo de regressão linear múltipla
modelo = sm.OLS(y,X_dummies).fit()

In [42]:
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  MEDIA   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                 5.274e+04
Date:                Sun, 02 Jul 2023   Prob (F-statistic):               0.00
Time:                        21:04:49   Log-Likelihood:            -1.3506e+07
No. Observations:             2344823   AIC:                         2.701e+07
Df Residuals:                 2344808   BIC:                         2.701e+07
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                             481.6038      0.375   1282.736      0.000     480.868     482.340
COMPUTADOR_INTERNET                42.7596      0.112    382.050      0.000      42.540      42.979
TP_ESCOLA_Privada                  11.7101      0.341     34.326      0.000      11.041      12.379
TP_ENSINO_Especial                -33.2234      1.143    -29.070      0.000     -35.463     -30.983
TP_DEPENDENCIA_ADM_ESC_Estadual   -21.8423      0.133   -163.740      0.000     -22.104     -21.581
TP_DEPENDENCIA_ADM_ESC_Federal     41.3721      0.376    110.068      0.000      40.635      42.109
TP_DEPENDENCIA_ADM_ESC_Privada     24.9192      0.332     74.997      0.000      24.268      25.570
TP_LOCALIZACAO_ESC_Rural          -15.9675      0.527    -30.281      0.000     -17.001     -14.934
EDUCACAO_PAI_MAE_A                -18.2692      0.591    -30.931      0.000     -19.427     -17.112
EDUCACAO_PAI_MAE_B                  4.1982      0.411     10.206      0.000       3.392       5.004
EDUCACAO_PAI_MAE_C                 14.3134      0.410     34.938      0.000      13.510      15.116
EDUCACAO_PAI_MAE_D                 20.0297      0.403     49.733      0.000      19.240      20.819
EDUCACAO_PAI_MAE_E                 33.7194      0.380     88.824      0.000      32.975      34.463
EDUCACAO_PAI_MAE_F                 59.6490      0.394    151.335      0.000      58.876      60.421
EDUCACAO_PAI_MAE_G                 69.4660      0.395    176.024      0.000      68.693      70.240
==============================================================================
Omnibus:                     9983.826   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11386.522
Skew:                          -0.112   Prob(JB):                         0.00
Kurtosis:                       3.258   Cond. No.                         29.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# solve problem: ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).#



